In [ ]:
import os
import json
import glob
import pandas as pd

from config import COUNTRIES, SEASONS, LABELS_DICT, LLM_MODEL_SUBNAME, \
                    SAVE_PLOTS, COMBINE_WEATHER_PLOTS, \
                    JSON_FILE_PATH, JSON_MASTER_FILE_PATH, \
                    WEATHER_RAW_PATH, WEATHER_COMBINED_PATH, WEATHER_PLOT_PATH, \
                    FAMILY_RAW_PATH, FAMILY_COMBINED_PATH, FAMILY_PLOT_PATH


print(f"LLM Model Selected in Config.py is {LLM_MODEL_SUBNAME}")
print("Save Plots: ", SAVE_PLOTS)
print("Combine Weather Plots: ", COMBINE_WEATHER_PLOTS)
print(JSON_FILE_PATH)
  
# Combine the JSON files for each country into one master JSON file  
combined_data = []
for country in COUNTRIES:
    # Read the JSON file
    output_json_file = JSON_FILE_PATH.replace("$COUNTRY$", country).replace(" ","-")
    
    with open(output_json_file, "r") as f:
        family_types_json = json.load(f)
    
    # Append the data to the combined list
    combined_data.extend(family_types_json)

# Save the combined data to a new file
with open(JSON_MASTER_FILE_PATH, "w") as f:
    json.dump(combined_data, f, indent=2)

print(f"Combined JSON saved to {JSON_MASTER_FILE_PATH}")

family_types_json = combined_data
# 'output_dir/Meta-Llama-3p1-405B-Instruct/data_l1_family_types/family_types_USA.json'

### Utility Functions

In [2]:
def create_dir(path, name):
    dir_path = os.path.join(path, name)
    os.makedirs(dir_path, exist_ok=True)
    return dir_path

def combine_csv(files):
    # Read and combine all CSV files for this family type
    combined_df = pd.DataFrame()
    for file in files:
        df = pd.read_csv(file)
        combined_df = pd.concat([combined_df, df], ignore_index=True)
    return combined_df

def create_family_directory(output_dir, country, family_type):
    family_dir = os.path.join(output_dir, country, family_type.replace(" ", "_"))
    os.makedirs(family_dir, exist_ok=True)
    return family_dir

def filter_files_by_season(files, season):
    return [file for file in files if season in file]

def validate_files(files, country, family_type, season):
    if len(files) != 2:
        print(f"Skipping {country} - {family_type} - {season}: Expected 2 files but found {len(files)}")
        return None, None

    weekday_file = next((file for file in files if "Weekday" in file), None)
    weekend_file = next((file for file in files if "Weekend" in file), None)
    
    if not weekday_file or not weekend_file:
        print(f"Missing Weekday or Weekend file for {country} - {family_type} - {season}")
        return None, None

    return weekday_file, weekend_file

### Get CSV Mapping

In [3]:
def get_csv_mapping(PROJ_PATH, family_types_json, process_type):    
    if process_type == "family":
        family_types = {entry["Country"]: [family["Family Type"] for family in entry["Families"]] for entry in family_types_json}
        
        # Extract countries and families
        countries = [entry["Country"] for entry in family_types_json]
        print("Available Countries:", countries)
        print("Family Types by Country:", family_types)

        # Get all CSV file paths in the folder
        csv_files = glob.glob(f"{PROJ_PATH}/*.csv")

        # Create a dictionary to map parsed JSON to CSV files
        csv_mapping = {}
        for country, families in family_types.items():
            csv_mapping[country] = {}
            for family_type in families:
                # Find matching CSV files for this country and family type
                family_type_clean = family_type.replace(' ', '-').replace("'", '-')
                matching_files = [
                    # file for file in csv_files if f"{country}_{family_type.replace(' ', '_')}" in file
                    file for file in csv_files if f"{country.replace(' ', '-')}_{family_type_clean}" in file
                ]
                csv_mapping[country][family_type] = matching_files

        # Check the mapping
        for country, families in csv_mapping.items():
            print(f"Country: {country}")
            for family, files in families.items():
                file_names = [os.path.basename(file) for file in files]
                print(f"  Family Type: {family} -> Files: {file_names}")
                
    elif process_type == "weather":        
        # Extract countries
        countries = [entry["Country"] for entry in family_types_json]
        print("Available Countries:", countries)
        print("Weather by Country")

        # Get all CSV file paths in the folder
        csv_files = glob.glob(f"{PROJ_PATH}/*.csv")

        # Create a dictionary to map parsed JSON to CSV files
        csv_mapping = {}
        for country in countries:
            # Find matching CSV files for this country
            matching_files = [
                file for file in csv_files if f"{country.replace(' ', '-')}" in file
                ]
            csv_mapping[country] = matching_files

        # Check the mapping
        for country, files in csv_mapping.items():
            file_names = [os.path.basename(file) for file in files]
            print(f"  Country: {country} -> Files: {file_names}")

    return csv_mapping

def combine_csvs(PROJ_PATH, csv_mapping, combine_type):  # combine_type: "family" or "weather"
    if not os.path.exists(PROJ_PATH):
        os.makedirs(PROJ_PATH)

    if combine_type == "family":
        for country, families in csv_mapping.items():
            # country_dir = create_dir(PROJ_PATH, country)
            for family_type, files in families.items():
                combined_df = combine_csv(files)
                if not combined_df.empty:
                    country_clean = country.replace(' ', '-')
                    family_type_clean = family_type.replace(' ', '-').replace("'", '-')
                    filename = f"{country_clean}_{family_type_clean}_combined.csv"
                    # file_path = os.path.join(country_dir, filename)
                    file_path = os.path.join(PROJ_PATH, filename)
                    combined_df.to_csv(file_path, index=False)
                    print(f"Combined CSV saved: {file_path}")

    elif combine_type == "weather":
        for country, files in csv_mapping.items():
            # country_dir = create_dir(PROJ_PATH, country)
            combined_df = combine_csv(files)
            if not combined_df.empty:
                filename = f"{country.replace(' ', '-')}_weather_combined.csv"
                # file_path = os.path.join(country_dir, filename)
                file_path = os.path.join(PROJ_PATH, filename)
                combined_df.to_csv(file_path, index=False)
                print(f"Combined CSV saved: {file_path}")

### Matplot Functions [Family]

In [4]:
import matplotlib.pyplot as plt
from ipywidgets import VBox, HBox, Dropdown, Button, Output
from IPython.display import display, clear_output

def plot_electricity_usage_matplotlib(df, title, show=True):
    # member_columns = [col for col in df.columns if col.endswith("_Consumption")]

    # Adjust member_columns to prioritize Heating_Consumption and Cooling_Consumption
    member_columns = [col for col in df.columns if col.endswith("_Consumption")]

    # Sort manually to prioritize Heating_Consumption and Cooling_Consumption
    member_columns = (
        ["Cooling_Consumption"] if "Cooling_Consumption" in member_columns else []
    ) + (
        ["Heating_Consumption"] if "Heating_Consumption" in member_columns else []
    ) + [col for col in member_columns if col not in {"Heating_Consumption", "Cooling_Consumption"}]

    if 'Hour' not in df.columns or 'Pattern' not in df.columns:
        raise ValueError("The DataFrame must contain 'Hour' and 'Pattern' columns.")

    df['Combined_Hour'] = range(len(df))

    unique_patterns = df['Pattern'].unique()
    if len(unique_patterns) < 2:
        raise ValueError("The DataFrame must contain at least two distinct 'Pattern' values for two days.")
    
    first_day_label = unique_patterns[0]
    second_day_label = unique_patterns[1]

    fig, ax = plt.subplots(figsize=(20, 6), dpi=300)

    df.set_index("Combined_Hour")[member_columns].plot(kind="bar", stacked=True, ax=ax)
    ax.axvline(23.5, color='gray', linestyle='--', linewidth=0.8)
    ax.text(12, -0.15 * ax.get_ylim()[1], first_day_label, ha="center", fontsize=16) 
    ax.text(36, -0.15 * ax.get_ylim()[1], second_day_label, ha="center", fontsize=16)

    # Update the legend to exclude the dashed line
    handles, labels = ax.get_legend_handles_labels()
    filtered_handles = handles[:len(member_columns)]  # Exclude extra handles (like the line)
    filtered_labels = [label.replace('_Consumption', '') for label in labels[:len(member_columns)]]
    ax.legend(filtered_handles, filtered_labels, loc='upper left', fontsize=14)

    # ax.set_title(title, fontsize=18)  # Increased font size for title
    ax.set_xlabel("Hour of the Day", fontsize=18)
    ax.set_ylabel("Electricity Usage (kW)", fontsize=18)
    ax.set_xticks(df['Combined_Hour'])
    ax.set_xticklabels([str(hour % 24) for hour in df['Combined_Hour']], rotation=0, fontsize=16)
    ax.tick_params(axis='y', labelsize=16)
    ax.grid(axis='y')
    plt.tight_layout()

    if show:
        plt.show()
        plt.close(fig)

def setup_interactive_matplotlib(csv_mapping):
    output = Output()
    
    # Initialize dropdowns
    initial_country = list(csv_mapping.keys())[0]  # Default to the first country in the mapping
    initial_families = list(csv_mapping[initial_country].keys()) if initial_country in csv_mapping else []
    
    country_dropdown = Dropdown(options=list(csv_mapping.keys()), description="Country:", value=initial_country)
    family_dropdown = Dropdown(options=initial_families, description="Family Type:", value=initial_families[0] if initial_families else None)
    season_dropdown = Dropdown(options=SEASONS, description="Season:")
    plot_button = Button(description="Plot")
    
    # Update family dropdown when country changes
    def update_family_dropdown(change):
        selected_country = change['new']
        families = list(csv_mapping.get(selected_country, {}).keys())
        family_dropdown.options = families
        if families:
            family_dropdown.value = families[0]  # Set default family type if available
        else:
            family_dropdown.value = None

    country_dropdown.observe(update_family_dropdown, names='value')

    # Handle plot button click
    def on_plot_click(_):
        selected_country = country_dropdown.value
        selected_family = family_dropdown.value
        selected_season = season_dropdown.value

        csv_files = csv_mapping.get(selected_country, {}).get(selected_family, [])
        matching_files = filter_files_by_season(csv_files, selected_season)
        weekday_file, weekend_file = validate_files(matching_files, selected_country, selected_family, selected_season)

        if not weekday_file or not weekend_file:
            with output:
                clear_output(wait=True)
                print(f"Invalid files for {selected_country} - {selected_family} - {selected_season}")
            return

        weekday_df = pd.read_csv(weekday_file)
        weekend_df = pd.read_csv(weekend_file)

        weekday_df['Pattern'] = 'Weekday'
        weekend_df['Pattern'] = 'Weekend'
        combined_df = pd.concat([weekday_df, weekend_df], ignore_index=True)

        with output:
            clear_output(wait=True)
            plot_electricity_usage_matplotlib(combined_df, f"{selected_country} - {selected_family} - {selected_season}")

    plot_button.on_click(on_plot_click)

    # Initial setup to populate the family dropdown based on the initial country
    update_family_dropdown({'new': initial_country})

    return VBox([HBox([country_dropdown, family_dropdown, season_dropdown, plot_button]), output])

def save_all_matplotlib_plots(csv_mapping, output_dir):
    for country, families in csv_mapping.items():
        for family_type, files in families.items():
            for season in SEASONS:
                # Filter files for the selected season
                matching_files = filter_files_by_season(files, season)
                weekday_file, weekend_file = validate_files(matching_files, country, family_type, season)
                
                if not weekday_file or not weekend_file:
                    continue

                # Load data
                weekday_df = pd.read_csv(weekday_file)
                weekend_df = pd.read_csv(weekend_file)

                # Combine data
                weekday_df['Pattern'] = 'Weekday'
                weekend_df['Pattern'] = 'Weekend'
                combined_df = pd.concat([weekday_df, weekend_df], ignore_index=True)

                # Save the plot
                family_dir = create_family_directory(output_dir, country, family_type)
                file_name = f"{country.replace(' ','-')}_{season}_Electricity_Usage_Breakdown"
                plot_electricity_usage_matplotlib(
                    combined_df, 
                    f"Electricity Usage for {family_type} in {country} during {season}", 
                    show=False
                )
                
                # Save as PNG with high resolution
                plt.savefig(os.path.join(family_dir, f"{file_name}.png"), dpi=300, bbox_inches="tight")
                # Save as PDF
                plt.savefig(os.path.join(family_dir, f"{file_name}.pdf"), bbox_inches="tight")
                plt.close()

### Matplot Functions [Weather]

In [5]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from ipywidgets import VBox, HBox, Dropdown, Button, Output
from IPython.display import display, clear_output

def setup_interactive_weather_plotting(csv_mapping, combined=True):
    output = Output()

    # Extract countries and seasons from the mapping
    countries = list(csv_mapping.keys())

    # Initialize dropdowns
    initial_country = countries[0]  # Default to the first country
    country_dropdown = Dropdown(options=countries, description="Country:", value=initial_country)
    season_dropdown = Dropdown(options=SEASONS, description="Season:", value=SEASONS[0])
    plot_button = Button(description="Plot")

    # Handle plot button click
    def on_plot_click(_):
        selected_country = country_dropdown.value
        selected_season = season_dropdown.value

        # Find the file matching the selected country and season
        csv_files = csv_mapping.get(selected_country, [])
        matching_files = [file for file in csv_files if selected_season in file]

        if not matching_files:
            with output:
                clear_output(wait=True)
                print(f"No data available for {selected_country} in {selected_season}")
            return

        try:
            # Load and plot the first matching file
            df = pd.read_csv(matching_files[0])
            value_columns = [col for col in df.columns if col.endswith('_Value')]

            if not value_columns:
                with output:
                    clear_output(wait=True)
                    print(f"No '_Value' columns found in the data for {selected_country} in {selected_season}")
                return

            with output:
                clear_output(wait=True)
                if combined:
                # Plot combined
                    plot_weather_data_matplotlib(df, f"{selected_country} - {selected_season}", combined=True)
                else:
                    # Plot individually for each column
                    for col in value_columns:
                        plot_weather_data_matplotlib(df, f"{selected_country} - {selected_season}", col=col, combined=False)
        except Exception as e:
            with output:
                clear_output(wait=True)
                print(f"Error loading or plotting file: {e}")

    plot_button.on_click(on_plot_click)

    return VBox([HBox([country_dropdown, season_dropdown, plot_button]), output])

def plot_weather_data_matplotlib(df, title, col=None, combined=False, show=True):
    value_columns = [c for c in df.columns if c.endswith('_Value')]

    if not value_columns:
        raise ValueError("No '_Value' columns found in the DataFrame.")

    if combined:
        # Combined plot logic
        num_cols = len(value_columns)
        rows = (num_cols + 2) // 3  # Calculate rows for 2x3 layout
        cols = min(num_cols, 3)

        fig, axes = plt.subplots(rows, cols, figsize=(15, 5 * rows), dpi=300)
        axes = axes.flatten()  # Flatten axes for easier iteration

        for idx, column in enumerate(value_columns):
            ax = axes[idx]
            ax.plot(df['Hour'], df[column], marker='o')
            ax.set_xlabel("Hour of the Day", fontsize=12)
            ax.set_ylabel(LABELS_DICT.get(column, column), fontsize=12)
            ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
            ax.tick_params(axis='x', labelsize=10)
            ax.tick_params(axis='y', labelsize=10)
            ax.set_xlim(0, 23)
            ax.set_xticks(range(0, 24, 2))  # Set ticks every 2 hours
            ax.grid(True)

        # Hide unused subplots if any
        for idx in range(num_cols, len(axes)):
            fig.delaxes(axes[idx])

        # plt.suptitle(title, fontsize=16, y=1.02)
        plt.tight_layout()

    else:
        # Individual plot logic
        if col is None:
            raise ValueError("Column name must be specified for individual plot.")

        plt.figure(figsize=(10, 6), dpi=300)
        plt.plot(df['Hour'], df[col], marker='o')
        # plt.title(f"{title}", fontsize=16)
        plt.xlabel("Hour of the Day", fontsize=14)
        plt.ylabel(LABELS_DICT.get(col, col), fontsize=14)
        plt.xticks(range(0, 24), fontsize=12)
        plt.yticks(fontsize=12)
        plt.xlim(0, 23)
        plt.grid(True)
        plt.tight_layout()

    if show:
        plt.show()
    # plt.close(fig)

def save_all_weather_plots(csv_mapping, output_dir, combined=True):
    for country, files in csv_mapping.items():
        for file in files:
            # Extract the season from the file name
            season = None
            for possible_season in SEASONS:
                if possible_season in file:
                    season = possible_season
                    break

            if season is None:
                print(f"Could not determine the season for file: {file}")
                continue

            # Load data
            try:
                df = pd.read_csv(file)
            except Exception as e:
                print(f"Error reading file {file}: {e}")
                continue

            value_columns = [col for col in df.columns if col.endswith('_Value')]

            if not value_columns:
                print(f"No '_Value' columns found in file: {file}")
                continue

            # Create directory for the country and season
            country_season_dir = os.path.join(output_dir, country, season)
            os.makedirs(country_season_dir, exist_ok=True)

            if combined:
                # Combined plot
                plot_weather_data_matplotlib(df, f"{country} - {season}", combined=True, show=False)
                file_name = f"{country}_{season}_Combined"
                plt.savefig(os.path.join(country_season_dir, f"{file_name}.png"), dpi=300, bbox_inches="tight")
                plt.savefig(os.path.join(country_season_dir, f"{file_name}.pdf"), bbox_inches="tight")
                plt.close()
            else:
                # Individual plots
                for col in value_columns:
                    plt.figure(figsize=(10, 6), dpi=300)
                    plt.plot(df['Hour'], df[col], marker='o')
                    # plt.title(f"{country} - {season} - {LABELS_DICT.get(col, col)}", fontsize=16)
                    plt.xlabel("Hour of the Day", fontsize=14)
                    plt.ylabel(LABELS_DICT.get(col, col), fontsize=14)
                    plt.xticks(range(0, 24), fontsize=12)
                    plt.yticks(fontsize=12)
                    plt.grid(True)
                    plt.tight_layout()
                    file_name = f"{country}_{season}_{col}"
                    plt.savefig(os.path.join(country_season_dir, f"{file_name}.png"), dpi=300, bbox_inches="tight")
                    plt.savefig(os.path.join(country_season_dir, f"{file_name}.pdf"), bbox_inches="tight")
                    plt.close()

### Weather and Family Consumption CSV Mapping

In [ ]:
csv_mapping_weather = get_csv_mapping(WEATHER_RAW_PATH, family_types_json, "weather")
combine_csvs(WEATHER_COMBINED_PATH, csv_mapping_weather, "weather")

In [ ]:
csv_mapping_family = get_csv_mapping(FAMILY_RAW_PATH, family_types_json, "family")
combine_csvs(FAMILY_COMBINED_PATH, csv_mapping_family, "family")

### Weather and Family Consumption Plots

In [ ]:
# Save plots for Matplotlib
if SAVE_PLOTS: save_all_weather_plots(csv_mapping_weather, WEATHER_PLOT_PATH, combined=COMBINE_WEATHER_PLOTS)

# Interactive Matplotlib
interactive_ui = setup_interactive_weather_plotting(csv_mapping_weather, combined=True)
display(interactive_ui)
print("\n\n")

In [ ]:
# Save plots for Matplotlib
if SAVE_PLOTS: save_all_matplotlib_plots(csv_mapping_family, FAMILY_PLOT_PATH)

# Interactive Matplotlib
matplotlib_ui = setup_interactive_matplotlib(csv_mapping_family)
display(matplotlib_ui)
print("\n\n")

### Plotly Functions [Family]

In [10]:
if False:
    import plotly.express as px
    import plotly.graph_objects as go
    from ipywidgets import VBox, HBox, Dropdown, Button, Output
    from IPython.display import display, clear_output

    PALETTE = px.colors.qualitative.T10  # Tableau T10 color palette

    def plot_electricity_usage_plotly(weekday_df, weekend_df, title, show=True):
        fig = go.Figure()

        # Add weekday data
        fig.add_trace(go.Bar(
            x=weekday_df["Hour"],
            y=weekday_df["Total_Electricity_Usage"],
            name="Weekday",
            marker_color=PALETTE[0]
        ))

        # Add weekend data
        fig.add_trace(go.Bar(
            x=weekend_df["Hour"],
            y=weekend_df["Total_Electricity_Usage"],
            name="Weekend",
            marker_color=PALETTE[1]
        ))

        fig.update_layout(
                title={
                    'text': title,
                    'x': 0.5,  # Center align the title
                    'xanchor': 'center',
                    'yanchor': 'top',
                    'font': {
                        'size': 24  # Increase title font size
                    }
                },
                xaxis_title="Hour",
                yaxis_title="Total Electricity Usage (kW)",
                xaxis=dict(
                    title=dict(font=dict(size=20)),  # Increase x-axis title font size
                    tickfont=dict(size=16)  # Increase x-axis tick font size
                ),
                yaxis=dict(
                    title=dict(font=dict(size=20)),  # Increase y-axis title font size
                    tickfont=dict(size=16)  # Increase y-axis tick font size
                ),
                barmode="group",
                xaxis_tickmode="linear",
                legend=dict(
                    font=dict(size=16)  # Increase legend font size
                )
            )

        if show:
            fig.show()
        
        return fig

    def setup_interactive_plotly(csv_mapping):
        output = Output()
        
        # Initialize dropdowns
        initial_country = list(csv_mapping.keys())[0]  # Default to the first country in the mapping
        initial_families = list(csv_mapping[initial_country].keys()) if initial_country in csv_mapping else []
        
        country_dropdown = Dropdown(options=list(csv_mapping.keys()), description="Country:", value=initial_country)
        family_dropdown = Dropdown(options=initial_families, description="Family Type:", value=initial_families[0] if initial_families else None)
        season_dropdown = Dropdown(options=SEASONS, description="Season:")
        plot_button = Button(description="Plot")
        
        # Update family dropdown when country changes
        def update_family_dropdown(change):
            selected_country = change['new']
            families = list(csv_mapping.get(selected_country, {}).keys())
            family_dropdown.options = families
            if families:
                family_dropdown.value = families[0]  # Set default family type if available
            else:
                family_dropdown.value = None

        country_dropdown.observe(update_family_dropdown, names='value')

        # Handle plot button click
        def on_plot_click(_):
            selected_country = country_dropdown.value
            selected_family = family_dropdown.value
            selected_season = season_dropdown.value

            csv_files = csv_mapping.get(selected_country, {}).get(selected_family, [])
            matching_files = filter_files_by_season(csv_files, selected_season)
            weekday_file, weekend_file = validate_files(matching_files, selected_country, selected_family, selected_season)

            if not weekday_file or not weekend_file:
                with output:
                    clear_output(wait=True)
                    print(f"Invalid files for {selected_country} - {selected_family} - {selected_season}")
                return

            weekday_df = pd.read_csv(weekday_file)
            weekend_df = pd.read_csv(weekend_file)

            with output:
                clear_output(wait=True)
                plot_electricity_usage_plotly(weekday_df, weekend_df, f"{selected_country} - {selected_family} - {selected_season}")

        plot_button.on_click(on_plot_click)

        # Initial setup to populate the family dropdown based on the initial country
        update_family_dropdown({'new': initial_country})

        return VBox([HBox([country_dropdown, family_dropdown, season_dropdown, plot_button]), output])

    def save_all_plotly_plots(csv_mapping, output_dir):
        for country, families in csv_mapping.items():
            for family_type, files in families.items():
                for season in SEASONS:
                    # Filter files for the selected season
                    matching_files = filter_files_by_season(files, season)
                    weekday_file, weekend_file = validate_files(matching_files, country, family_type, season)
                    
                    if not weekday_file or not weekend_file:
                        continue

                    # Load data
                    weekday_df = pd.read_csv(weekday_file)
                    weekend_df = pd.read_csv(weekend_file)

                    # Save the plot
                    family_dir = create_family_directory(output_dir, country, family_type)
                    file_name = f"{country.replace(' ','-')}_{season}_Electricity_Usage"
                    fig = plot_electricity_usage_plotly(
                        weekday_df, 
                        weekend_df, 
                        f"Electricity Usage for {family_type} in {country} during {season}",
                        show=False
                    )
                    # html_path = os.path.join(family_dir, f"{file_name}.html")
                    # fig.write_html(html_path)

                    png_path = os.path.join(family_dir, f"{file_name}.png")
                    fig.write_image(png_path, width=1280, height=720)

                    del fig

In [11]:
if False:
    # Save plots for Plotly
    if SAVE_PLOTS: save_all_plotly_plots(csv_mapping_family, FAMILY_PLOT_PATH)

    # # Interactive Plotly
    plotly_ui = setup_interactive_plotly(csv_mapping_family)
    display(plotly_ui)
    print("\n\n")